In [4]:
!pip install lxml

In [10]:
#import the necessary packages
from bs4 import BeautifulSoup
import requests
import pandas as pd
import urllib
from sklearn.cluster import KMeans

In [11]:
#Parse the URL using BeautifulSoup
url=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(url, 'lxml')
df_table=soup.find('table',{'class:','wikitable sortable'})
columns=['PostalCode','Borough','Neighborhood']
dataframe_flattened=pd.DataFrame(columns=columns)
rows=df_table.find_all('tr')
rows.pop(0)

<tr>
<th>Postal code
</th>
<th>Borough
</th>
<th>Neighborhood
</th></tr>

# Part 1 : Setting up data frame

In [26]:
#append the data in the dataframe
for row in rows:
    tds=row.find_all('td')
    if tds[1].get_text().strip()!='Not assigned':
        dataframe_flattened=dataframe_flattened.append({'PostalCode':tds[0].get_text(),'Borough':tds[1].get_text(),'Neighborhood':tds[1].get_text() if tds[2].get_text().rstrip()=='Not assigned' else tds[2].get_text().rstrip()},ignore_index=True)

In [38]:
df=dataframe_flattened.groupby(['PostalCode','Borough']).apply(lambda x:','.join(x['Neighborhood'])).to_frame(name='Neighborhood').reset_index()

In [39]:
df["PostalCode"] = df["PostalCode"].str.replace("\n","")
df["Borough"] = df["Borough"].str.replace("\n","")
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern / Rouge,Malvern / Rouge"
1,M1C,Scarborough,"Rouge Hill / Port Union / Highland Creek,Rouge..."
2,M1E,Scarborough,"Guildwood / Morningside / West Hill,Guildwood ..."
3,M1G,Scarborough,"Woburn,Woburn"
4,M1H,Scarborough,"Cedarbrae,Cedarbrae"


In [30]:
df.shape

(103, 3)

# Part 2 : The latitude and the longitude coordinates

In [31]:
df_cord=pd.read_csv("https://cocl.us/Geospatial_data") 
df_cord.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df_cord.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [40]:
result=pd.merge(df,df_cord[['PostalCode','Latitude','Longitude']],on='PostalCode')
result.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern / Rouge,Malvern / Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill / Port Union / Highland Creek,Rouge...",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood / Morningside / West Hill,Guildwood ...",43.763573,-79.188711
3,M1G,Scarborough,"Woburn,Woburn",43.770992,-79.216917
4,M1H,Scarborough,"Cedarbrae,Cedarbrae",43.773136,-79.239476
5,M1J,Scarborough,"Scarborough Village,Scarborough Village",43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park / Ionview / East Birchmount Park,...",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile / Clairlea / Oakridge,Golden Mile ...",43.711112,-79.284577
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village W...,43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff / Cliffside West,Birch Cliff / Cli...",43.692657,-79.264848


# Part 3 : Explore and cluster the neighborhoods in Toronto

In [41]:
neigh_toronto=result[result['Borough'].str.contains('Toronto')]
neigh_toronto.head(30)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,"The Beaches,The Beaches",43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West / Riverdale,The Danforth Wes...",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar / The Beaches West,India Bazaar /...",43.668999,-79.315572
43,M4M,East Toronto,"Studio District,Studio District",43.659526,-79.340923
44,M4N,Central Toronto,"Lawrence Park,Lawrence Park",43.728020,-79.388790
45,M4P,Central Toronto,"Davisville North,Davisville North",43.712751,-79.390197
46,M4R,Central Toronto,"North Toronto West,North Toronto West",43.715383,-79.405678
47,M4S,Central Toronto,"Davisville,Davisville",43.704324,-79.388790
48,M4T,Central Toronto,"Moore Park / Summerhill East,Moore Park / Summ...",43.689574,-79.383160
49,M4V,Central Toronto,Summerhill West / Rathnelly / South Hill / For...,43.686412,-79.400049


## Exploring near by venues

In [43]:
CLIENT_ID = 'DNP3VAQQR25JMWPET1RJHZMIOLSARCOQP13TQPCUTETR3VNU' # your Foursquare ID
CLIENT_SECRET = 'VWI5AT3UPRTMUIBBX45ZHTRBDE0BZZU5VNNFILZLKYY1QMU1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT=100
toronto_venues = getNearbyVenues(names=neigh_toronto['Neighborhood'],
                                   latitudes=neigh_toronto['Latitude'],
                                   longitudes=neigh_toronto['Longitude']
                                  )
toronto_venues.head()

The Beaches,The Beaches
The Danforth West / Riverdale,The Danforth West / Riverdale
India Bazaar / The Beaches West,India Bazaar / The Beaches West
Studio District,Studio District
Lawrence Park,Lawrence Park
Davisville North,Davisville North
North Toronto West,North Toronto West
Davisville,Davisville
Moore Park / Summerhill East,Moore Park / Summerhill East
Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park,Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park
Rosedale,Rosedale
St. James Town / Cabbagetown,St. James Town / Cabbagetown
Church and Wellesley,Church and Wellesley
Regent Park / Harbourfront,Regent Park / Harbourfront
Garden District / Ryerson,Garden District / Ryerson
St. James Town,St. James Town
Berczy Park,Berczy Park
Central Bay Street,Central Bay Street
Richmond / Adelaide / King,Richmond / Adelaide / King
Harbourfront East / Union Station / Toronto Islands,Harbourfront East / Union Station / Toronto Islands
Toronto Dominion Centre / 

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"The Beaches,The Beaches",43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,"The Beaches,The Beaches",43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,"The Beaches,The Beaches",43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,"The Beaches,The Beaches",43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West / Riverdale,The Danforth Wes...",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [44]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighborhood']=toronto_venues['Neighborhood']

In [45]:
toronto_onehot.columns = toronto_onehot.columns.get_level_values(0)
toronto_grouped=toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Post Office,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,"Berczy Park,Berczy Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.017241,0.000000,0.0,0.0,0.0,0.000000,0.017241,0.0,0.017241,0.034483,0.0,0.000000,0.0,0.017241,0.017241,0.0,0.034483,0.0,0.0,0.017241,0.0,0.000000,0.0,0.000000,0.0,0.017241,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.017241,0.034483,0.0,0.0,0.0,0.034483,0.0,0.0,0.0,0.000000,0.017241,0.051724,0.068966,0.0,0.0,0.0,0.0,0.0,0.017241,0.000000,0.017241,0.000000,0.0,0.017241,0.0,0.017241,0.0,0.0,0.0,0.0,0.017241,0.0,0.017241,0.000000,0.0,0.0,0.0,0.000000,0.017241,0.0,0.0,0.0,0.000000,0.017241,0.000000,0.0,0.0,0.017241,0.0,0.0,0.0,0.0,0.0,0.0,0.017241,0.017241,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0000

## top 10 most common venues for a neighborhood

In [46]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [47]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Berczy Park,Berczy Park",Coffee Shop,Cocktail Bar,Seafood Restaurant,Café,Cheese Shop,Beer Bar,Bakery,Restaurant,Park,Fish Market
1,Brockton / Parkdale Village / Exhibition Place...,Café,Breakfast Spot,Coffee Shop,Gym,Furniture / Home Store,Intersection,Bar,Bakery,Restaurant,Climbing Gym
2,"Business reply mail Processing Centre,Business...",Pizza Place,Garden Center,Burrito Place,Auto Workshop,Fast Food Restaurant,Farmers Market,Garden,Spa,Brewery,Park
3,CN Tower / King and Spadina / Railway Lands / ...,Airport Lounge,Airport Service,Airport,Boat or Ferry,Harbor / Marina,Coffee Shop,Rental Car Location,Sculpture Garden,Boutique,Plane
4,"Central Bay Street,Central Bay Street",Coffee Shop,Café,Italian Restaurant,Sandwich Place,Ice Cream Shop,Thai Restaurant,Salad Place,Bar,Burger Joint,Bubble Tea Shop


## K Means on dataset

In [48]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [49]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neigh_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,M4E,East Toronto,"The Beaches,The Beaches",43.676357,-79.293031,2,Health Food Store,Trail,Pub,Yoga Studio,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop
41,M4K,East Toronto,"The Danforth West / Riverdale,The Danforth Wes...",43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Bookstore,Frozen Yogurt Shop,Ice Cream Shop,Yoga Studio,Spa,Japanese Restaurant
42,M4L,East Toronto,"India Bazaar / The Beaches West,India Bazaar /...",43.668999,-79.315572,0,Fast Food Restaurant,Park,Gym,Pizza Place,Board Shop,Brewery,Sandwich Place,Burrito Place,Restaurant,Pub
43,M4M,East Toronto,"Studio District,Studio District",43.659526,-79.340923,0,Café,Coffee Shop,Bakery,Gastropub,American Restaurant,Brewery,Yoga Studio,Fish Market,Italian Restaurant,Convenience Store
44,M4N,Central Toronto,"Lawrence Park,Lawrence Park",43.728020,-79.388790,4,Park,Swim School,Bus Line,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop


In [50]:


from geopy.geocoders import Nominatim
address = 'Toronto, Ontario,Canada'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))



The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [51]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Solving environment: done

# All requested packages already installed.

